In [12]:
import ctypes
import ctypes.util
import numpy
import time
import math
import h5py
import warnings
import os
import os.path
import errno
import sys
import scipy.ndimage
import warnings
import gc
import nidaqmx
import os
import os.path
from os import path

import ctypes
import ctypes.util
import numpy
import time
import math
import camera.hamamatsu_camera as hc
import rs232.RS232 as RS232
import filter_wheel.fw102c_LB as fw102c_LB
import laser.obis as obis
import laser.sapphire as sapphire
import xyz_stage.ms2000 as ms2000
# import thorlabs_apt as apt
import generator.ni_LB as generator
# import utils.findbounds as findbounds
import utils.utils as utils
import utils.background as background
import h5py
import warnings
import os
import os.path
import errno
import sys
import scipy.ndimage
import warnings
import gc
import nidaqmx
from os import path
import shutil
import logging

In [3]:
############# SCAN PARAMETERS #############
sys.path.append('E')
drive = 'E'
fname = '10-8-20_20X_OTLS3_Imaizumi_TDE146_488561' # specimen names
copy_to_Zdrive = 'no'
xMin = 2.90 #
xMax = 6.56 # #needs to be at least 2mm i think?
#ycenter = -26.67
yMin = -7.76 # #ycenter - .385
yMax = -5.30#-5.68 #-5.28 #-4.30##ycenter +.7
zMin = -3.31 # #zcenter - .01
zMax = -3.15 # #zcenter + .01 #if these numbers do not make sense, might get error "idx was referenced before assignment" #if these numbers do not make sense, might get error "idx was referenced before assignment"

# #for ECi
# #Dimensions of FOV
# xWidth = 0.209 #in um, not mm (2nd gen system 0.48) 300um/1427px
# yWidth = 0.385 # mm camera's horizontal FOV
# zWidth = 0.03 #3 # mm calculated based on 610px/125um 11-20-19(2nd gen system 0.07)this will directly change how far z-strips are placed from each otehr in big stitcher
# # LB added lines below 1/4/20: User-defined spacing between image tiles to optimize stitching. by default they should be xWidth, yWidth, and zWidth
# yStitchOverlay = 0.3825 #User-defined spacing between image tiles horizontally, will affect XML file but not imaging
# zStitchOverlay = .03 #User-defined spacing between image tiles vertically, will affect XML file but not imaging

#for n = 1.46
xWidth = 0.2212 #in um measure by moving stage in Y direction (300um stage movement per 1352 px) 1808px 400um
yWidth = .45 #.453 #in mm camera's horizontal FOV
zWidth = .0395#.0401 #in mm camera's vertical spacing, i.e. (um/px)*(256px)/sqrt(2)
yStitchOverlay = .447 #User-defined spacing between image tiles horizontally, will affect XML file but not imaging
zStitchOverlay = zWidth #User-defined spacing between image tiles vertically, will affect XML file but not imaging


# #for Clearsee
# xWidth = 0.22779
# yWidth = .467
# zWidth = .04123
# yStitchOverlay = .46 #User-defined spacing between image tiles horizontally, will affect XML file but not imaging
# zStitchOverlay = .03 #User-defined spacing between image tiles vertically, will affect XML file but not imaging

camY = 256 # pixels #"Z" direction, vertical in camera's FOV
camX = 2048 # pixels "Y" direction, horizontal in camera's FOV
expTime = 25.0 #4.99 ms
camOffset = 0.0 # counts
wavelengths = numpy.array([488,561]) # lambda in nm
laser_powers = numpy.array([3,2]) #filter 1: 685+ filter 2: LP + single-band filter collects 495-552nm
galvoXoffset = 2.84 #in V
galvoYoffset = numpy.array([-.95,-.94]) #for 660nm, 488nm respectively. usually these numbers should be the same for 20X imaging

galvoXamp = 1.5 # in V. should be 1.5V for 20X imaging
galvofreq = 1000 #in Hz. rule of thumb should tune to whatever minimum frequency looks "smooth" on the live camera feed
binning = '2x2'
flatField = 0

######### PARAMETERS ###########
xMin = xMin - .05 #extra room for safety
xMax = xMax + .05
#y min = ymin-ywidth/2 and ymax = ymax + ywidth makes it so that it'll tile 1 tile to the left and right of your center
yMin = yMin #- yWidth/2 #to account for FOV/2 width not accounted for when estimating tissue bounds, + a little extra
yMax = yMax #+ yWidth/2

######### INITIALIZE PARAMETERS ###########
xLength = xMax - xMin # mm
yLength = yMax - yMin #LB commented math.ceil((yMax - yMin)/yWidth)*yWidth # mm
zLength = math.ceil((zMax - zMin)/zWidth)*zWidth # mm
volume = xLength*yLength*zLength
#Goal with yTiles is to cover whole area that user entered, + image extra if necessary
yTiles = int(math.ceil(yLength/yWidth)) #previously int(round(yLength/yWidth))
zTiles = int(round(zLength/zWidth))
nTiles = zTiles*yTiles*len(wavelengths)
print('tissue volume = ' + str(volume) + 'mm^3')
print('number of tiles = ' + str(nTiles))
print('folder name = ' + str(fname))
xOff = xMax - (xLength)/2
yOff = yMax - yLength/2
zOff = zMin
nFrames = int(round(xLength/(xWidth/1000))) #number of frames in X direction
###want to make sure that structure at center of coordinates will be imaged at center of FOV

if binning == '1x1':
    binFactor = 1
elif binning == '2x2':
    binFactor = 2
elif binning == '4x4':
    binFactor = 4
else:
    binFactor = 1


# print('Y positions will be = ')
for j in range(zTiles): 
    for k in range(yTiles):
        for i in range(len(wavelengths)):
            idx = k+j*yTiles+i*yTiles*zTiles
            # if idx == 232:
            yPos = yOff- yLength/2.0 + k*yWidth #+ yWidth/2.0
#             print('y = ' + str(yPos))
            zPos = j*zWidth + zOff
            # print('z = ' + str(zPos))
print('nFrames = ' + str(nFrames))
scantime_singlestrip = xLength/(xWidth*binFactor/expTime)
scantime_theoreticalmaxspeed = xLength/(xWidth*binFactor/1.25)
print('estimated time for imaging = ' + str((scantime_singlestrip+60)*nTiles/3600) + 'hrs')
# print('theoretical imaging time (max speed) = ' + str((scantime_theoreticalmaxspeed)*nTiles/60) + 'minutes')


tissue volume = 1.8267959999999996mm^3
number of tiles = 60
folder name = 10-8-20_20X_OTLS3_Imaizumi_TDE146_488561
nFrames = 16998
estimated time for imaging = 4.541289933694996hrs


In [ ]:
# lsmfx_LB.scan3D(drive, fname, xOff, yOff, zOff, xLength, yLength, zLength, xWidth, yWidth, zWidth, yStitchOverlay, zStitchOverlay, camY, camX, expTime, binning, wavelengths, laser_powers, galvoXoffset, galvoXamp, galvofreq, galvoYoffset, camOffset, flatField)


In [ ]:
# def scan3D(drive, save_dir, xoff, yoff, zoff, xLength, yLength, zLength, xWidth, yWidth, zWidth, yStitchOverlay, zStitchOverlay, camY, camX, expTime, binning, wavelengths, laser_powers,  galvoXoffset, galvoXamp, galvofreq, galvoYoffset, camOffset, flatField = 0, vis = 0):

In [37]:
save_dir = fname
xoff = xOff
yoff = yOff
zoff = zOff

## lsmfx

In [4]:
if binning == '1x1':
    binFactor = 1
elif binning == '2x2':
    binFactor = 2
elif binning == '4x4':
    binFactor = 4
else:
    binFactor = 1

if wavelengths.size > 1:
    nWavelengths = len(wavelengths)
else:
    nWavelengths = 1

In [5]:
xWidth = xWidth*binFactor
camY = int(camY/binFactor)
camX = int(camX/binFactor)
nFrames = int(round(xLength/(xWidth/1000))) #number of frames in X direction
#nFrames = 2048
yTiles = int(math.ceil(yLength/yWidth))  #int(round(yLength/yWidth))
zTiles = int(round(zLength/zWidth))
scanSpeed = xWidth/expTime #um/ms or mm/sec # xWidth/(1.0/(1.0/(expTime/1000.0))*1000) LB modified to be simpler 11-22-19
print('scanSpeed = ' + str(scanSpeed))

print('number of frames in X direction = ' + str(nFrames))
print('number of tiles in Y direction = ' + str(yTiles))
print('number of tiles in Z direction = ' + str(zTiles))

scanSpeed = 0.017696
number of frames in X direction = 8499
number of tiles in Y direction = 6
number of tiles in Z direction = 5


In [13]:
fWheel = fw102c_LB.FW102C(baudrate = 115200, port = 'COM6')

hcam = hc.HamamatsuCameraMR(camera_id=0)
print(hcam)

# Set camera properties
hcam.setPropertyValue("defect_correct_mode", "OFF") # keep defect mode on
hcam.setPropertyValue("readout_speed", 2) # 1 or 2. 2 is fastest mode
hcam.setPropertyValue("exposure_time", expTime/1000.0) # convert from msec to sec
hcam.setPropertyValue("subarray_hsize", camX)
hcam.setPropertyValue("subarray_vsize", camY)
hcam.setPropertyValue("subarray_vpos", 1024-camY/2)
hcam.setPropertyValue("binning", binFactor)

# Set trigger properties
hcam.setPropertyValue("trigger_source", 'INTERNAL') # 1 (internal), 2 (external), 3 (software)
hcam.setPropertyValue("trigger_mode", 'START') # 1 (normal), 6 (start)
hcam.setPropertyValue("trigger_active", 'EDGE') # 1 (edge), 2 (level), 3 (syncreadout)
hcam.setPropertyValue("trigger_polarity", 'POSITIVE') # 1 (negative), 2 (positive)

hcam.setACQMode("fixed_length", nFrames)

############# SETUP XYZ STAGE #############

xyzStage = ms2000.MS2000(baudrate = 9600, port = 'COM5')
xyzStage.setScanF(1)
xyzStage.setBacklash(0)
xyzStage.setTTL(0)
initialPos = xyzStage.getPosition()
print(xyzStage)

 unknown property name: readout_speed
max camera backlog was: 0


In [18]:
waveform = generator.waveformGenerator(freq = galvofreq, Xamplitude = galvoXamp, Xoffset = galvoXoffset, Yamplitude = 0, Yoffset = galvoYoffset[0])

In [14]:
imgShape = (nFrames, camY, camX) 
chunkSize1 = 256/binFactor
chunkSize2 = 32/binFactor #for Z (256px) direction
chunkSize3 = 256/binFactor
write_threads = []
im = numpy.zeros((nFrames, camY, camX), dtype = 'uint16') #initialize: "X" (scan) direction, Z (256px) direction, Y direction


In [16]:
f = h5py.File('E://10-8-20_20X_OTLS3_Imaizumi_TDE146_488561//data.h5','r+')

In [22]:
laser = sapphire.SapphireLaser(com = 'COM13')
waveform.adjust_Yoffset(galvoYoffset[i])
fWheel.setPosition(561)

In [53]:
idx_LB = 0
for j in range(zTiles): 
    for k in range(yTiles): 
        for i in range(nWavelengths):
            idx = k+j*yTiles+i*yTiles*zTiles
            idx_tile = k+j*yTiles
            idx_channel = i
            
            if idx == 59: #messed up tile, the only one we will write to
                ## wavelengths[i] = 561 for this tile
#                 resgroup = f.create_group('/t00000/s' + str(idx).zfill(2) + '/' + str(0))
#                 data = f.require_dataset('/t00000/s' + str(idx).zfill(2) + '/' + str(0) + '/cells', chunks = (chunkSize1, chunkSize2, chunkSize3), dtype = 'int16', shape = imgShape, compression = 32016, compression_opts=(round(2*1000), 1, round(2.1845*1000), 0, round(1.6*1000)))
                # GO TO INITIAL POSITIONS
#                 moveSpeed = .4
#                 xyzStage.setVelocity('X',moveSpeed) #used to be .5
#                 xyzStage.setVelocity('Y',moveSpeed) #used to be .5
#                 xyzStage.setVelocity('Z',moveSpeed) #used to be .5

#                 xPos = xLength/2.0 - xoff
#                 yPos = -yLength/2.0+k*yWidth + yoff #COMMENTED OUT LB +yWidth/2.0
#                 zPos = j*zWidth + zoff
#                 print('Starting tile ' + str(idx_LB+1) + '/' + str(zTiles*yTiles*nWavelengths)) #LB modified for indexing
#                 ts = time.gmtime()
#                 print(time.strftime("%x %X", ts))
#                 print('move x position to: ' + str(-xPos)+ ' mm') #modified LB 11-17-19, used to be str(-xPos-.035)
#                 print('move y position to: ' + str(yPos)+ ' mm')
#                 print('move z position to: ' + str(zPos)+ ' mm')

#                 xyzStage.goAbsolute('X', -xPos, False) #used to be -xPos - .035
#                 xyzStage.goAbsolute('Y', yPos, False)
#                 xyzStage.goAbsolute('Z', zPos, True)

#                 if scanSpeed > 0.5:
#                     raise Exception('Scan speed has been set to > 0.5mm/s! This could cause sample movement')

#                 xyzStage.setVelocity('X',scanSpeed)
#                 xyzStage.setScanR(-xPos, -xPos + xLength)
#                 xyzStage.setScanV(yPos) 

#                 #TURN LASER ON
#                 laser.setPower(laser_powers[i]) #power in mW
#                 laser.turnOn() #turn laser on

#                 # START SCAN
#                 hcam.startAcquisition() #start camera acquisition
#                 xyzStage.scan(False,laser,-xPos) #laser turns off laser as soon as scan is done, and others commeand...
#                 #stage to go back to original position as soon as scan is done at scanSpeed. modified LB 1-3-29 and 2-3-20

                # CAPTURE IMAGES
                count_old = 0
                count_new = 0
                count = 0
                
                if flatField == 1:
                    print('flatfield shoudl not be 1')
                    
                    
                else: #if flat field correction = 0
                    #after this loop, im.shape = (direction of X scan, direction of vertical FOV, direction of horizontal FOV) i.e. (X, Z, Y)
                    while count < nFrames-1:
                        time.sleep(0.01)
                        print('getting frames')
#                         [frames, dims] = hcam.getFrames()
                        count_old = count
                        # Save frames.
                        for aframe in frames:
#                             np_data = aframe.getData()
#                             im[count] = numpy.reshape(np_data, (camY, camX)) #orient so camera's vertical FOV is row direction
                            im[count] = numpy.ones((camY,camX), dtype = 'uint16')
                            count += 1
                        count_new = count
                        if count_new == count_old:
                            count = nFrames #reached last frame in scan
                        print(str(count_new) + '/' + str(nFrames) + ' frames collected...')
                        #ts = time.gmtime()
                        #print(time.strftime("%x %X", ts))
                        data[count_old:count_new] = im[count_old:count_new].astype('int16') # added ' .astype('int16') ' LB 10-4-20
                print('im.shape = ' + str(im.shape))
                print('writing downsampled resolutions')
                if idx_LB > 0: #
                    previous_thread = write_threads[idx_LB-1]
                    while previous_thread.alive() == True:
                        time.sleep(0.1)

                current_thread = utils.writeBDV(f, im, idx, binFactor)
                write_threads.append(current_thread)
                current_thread.start()

                if idx == (nWavelengths*yTiles*zTiles-1):
                    current_thread.join()


                # WRITE VISUALIZATION FILE
                print('writing visualization file')
                print('im.shape = ' + str(im.shape))
                print('idx = ' + str(idx))

#                 hcam.stopAcquisition()
#                 gc.collect()
                idx_LB += 1
                
print('idx = ' + str(idx))
print('idx_tile = ' + str(idx_tile))
print('idx_channel = ' + str(idx_channel))
print('channels = ' + str(nWavelengths))

# WRITES WHOLE FILE TO XML
utils.write_xml(drive = drive, save_dir = save_dir, idx = idx, idx_tile = idx_tile, idx_channel = idx_channel, channels = nWavelengths, tiles_y = yTiles, tiles_z = zTiles, sampling = xWidth, binning = binFactor, offset_y = yStitchOverlay, offset_z = zStitchOverlay, x = imgShape[0], y = imgShape[1], z = imgShape[2])

fWheel.shutDown()
laser.shutDown()
waveform.stop() #stops galvo
hcam.shutdown()
f.close()

xyzStage.setVelocity('X',0.5)
xyzStage.setVelocity('Y',0.5)
xyzStage.setVelocity('Z',0.5)			

xyzStage.goAbsolute('X', initialPos[0], False)
xyzStage.goAbsolute('Y', initialPos[1], False)
xyzStage.goAbsolute('Z', initialPos[2], True)

xyzStage.shutDown()


getting frames
8499/8499 frames collected...
im.shape = (8499, 128, 1024)
writing downsampled resolutions
writing visualization file
im.shape = (8499, 128, 1024)
idx = 59
idx = 59
idx_tile = 29
idx_channel = 1
channels = 2
Writing BigDataViewer XML file...


In [52]:
frames

 ...]